In [ ]:
#pip install moviepy pygame 
import os, re, math, pandas as pd
from matplotlib import pyplot as plt, ticker as ticker, font_manager 

#https://zulko.github.io/moviepy/getting_started/quick_presentation.html
from moviepy.editor import VideoClip, AudioFileClip, VideoFileClip
from moviepy.video.io.bindings import mplfig_to_npimage

In [ ]:
old_families = set(font_manager.get_font_names())
font_files = font_manager.findSystemFonts(fontpaths=['font']) 
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)   
new_families = set(font_manager.get_font_names())
#print(new_families.difference(old_families)) 

# I will not disclose my font files but I got this 
# particular font "Source Han Sans CN" from 
# https://freefonts.top/font/60977eb18f0305e19313ecf2
plt.rcParams['font.family'] = "Source Han Sans CN" 
plt.rcParams['font.size'] = 22 

re_com = re.compile('[0-9 .]*')
def title_to_vector(title):
    #title = '433. 做一期播客，要如何寻找需要的信息？'
    re_match = re_com.match(title)
    first_line = '梁文道 八分 '+re_match.group()
    following_lines = title[re_match.end():]
    num_row = len(following_lines) // 10 + 2 
    ys = [0.2*i for i in range(num_row)]
    median = sum(ys)/len(ys)
    ys = [0.5+i-median for i in ys]
    ys = sorted(ys,reverse=True)
    ts = [first_line]
    for i in range(num_row-1):
        ts.append(following_lines[i*10:(i+1)*10]) 
    return ys, ts 

my_dpi = 96
text_kwargs = dict(ha='center', va='center', color='black')
shape = (320, 240)

def make_frame(_time): # run for every frame
    ax.clear()
    ax.axis('off') 
    #ys, title_in_pieces = 
    for y, piece in zip(*title_to_vector(title)):
        ax.text(0.5, y, piece, **text_kwargs) 
    return mplfig_to_npimage(fig) 

In [ ]:
df_audio = pd.read_csv('audio/list.csv')
df_audio = df_audio.set_index('vid')
df_audio = df_audio[df_audio['t_end'].notna()]
df_audio

In [ ]:
fig, ax = plt.subplots(figsize=(shape[0]/my_dpi, shape[1]/my_dpi), dpi=my_dpi)
ax.xaxis.set_major_locator(ticker.NullLocator())
ax.yaxis.set_major_locator(ticker.NullLocator()) 
for vid, row in df_audio.iterrows(): 
    audio_file=vid+'.mp4'
    title   = row['title']
    t_end   = row['t_end'] 
    if not os.path.exists('audio/'+audio_file):
        print('[Original Audio File Does Not Exist]', audio_file)
        continue
    if os.path.exists('video/'+audio_file):
        print('[Edited Video File Already Exists]', audio_file)
        continue 
    print('[Yes Audio, No Video]', audio_file, title, t_end)
    audioClip = AudioFileClip('audio/'+audio_file)
    audioClip = audioClip.subclip(t_end=t_end)
    videoClip = VideoClip(make_frame, duration=audioClip.duration).set_audio(audioClip)
    #videoClip.ipython_display(fps=1, loop=True, autoplay=True)
    videoClip.write_videofile('video/'+audio_file, fps=1) 